In [14]:
!pip install -q langchain==0.2.2
!pip install -q langchain-openai==0.1.8

In [15]:
import os

In [16]:
os.environ["OPENAI_API_KEY"] = ""

In [17]:
from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage, SystemMessage
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser, JsonOutputParser

In [18]:
from langchain.globals import set_debug
set_debug(False)

<hr>

### Mockup Email

In [19]:
Frontend_email = """
Dear Hiring Team,
My name is Kasikorn Rakthai, and I am excited to apply for the Frontend Developer position at KBTG. With a Bachelor’s degree in Computer Science from Chulalongkorn University and over three years of experience in frontend development, I am confident in my ability to contribute to your innovative team.
I am proficient in HTML, CSS, JavaScript, and frameworks such as React.js and Angular. At [Your Current or Previous Company], I successfully led a website redesign that increased user engagement by 30% and improved conversion rates by 20%.
KBTG's commitment to technological innovation greatly excites me, and I am eager to bring my skills to your team. Attached is my resume for your review. I am available for an interview at your convenience and can be reached at 089-123456.
Thank you for considering my application.
Best regards,
Kasikorn Rakthai
"""
chef_email = """
Dear Hiring Manager,
I am writing to express my interest in the Head Chef position at KBTG. With over 15 years of culinary experience, including 5 years as Executive Chef at renowned restaurants, I am confident in my ability to contribute to your esteemed establishment.
My expertise in diverse cuisines, team leadership, and innovative menu development align well with the vision of KBTG. I am excited about the opportunity to bring my passion for culinary excellence to your team.
Please find my resume attached for your review. I look forward to the possibility of discussing how my background, skills, and certifications can be an asset to KBTG.
Thank you for considering my application.
Sincerely,
Aroii Dee
"""


<hr>

## 01 Email data extraction

In [54]:
prompt_template = ChatPromptTemplate.from_messages(
    [
        ("system",
                """
                You are a Hiring Manager at KBTG reading a candidate's email.
                A candidate's email text will be given.
                Your task is to identify the following information and convert it into JSON object with these keys:
                - job_position : string, the job position which the candidate is applying for.
                - company_name : string, the company name which the candidate is applying for.
                - candidate_name : string, the candidate name
                Answer in the form of a JSON object
                """),
        ("user",
                "{email}"
                )

    ]
)
model = ChatOpenAI(
    model = "gpt-3.5-turbo",
    temperature = 0.0
)
# parser = StrOutputParser()
parser = JsonOutputParser()

chain = prompt_template | model | parser

In [55]:
chain.invoke(
    {"email":Frontend_email}
)

{'job_position': 'Frontend Developer',
 'company_name': 'KBTG',
 'candidate_name': 'Kasikorn Rakthai'}

<hr>

## 02 Pass or NotPass

In [79]:
def clean(text):
    return text.lower().strip().replace(" ","")
def make_decision(candidate_data):
    opening_positions = ["Frontend Developer","Backend Developer","AI Developer","Blockchain Develop"]
    opening_positions = set([clean(e) for e in opening_positions])
    critiria_1 = clean(candidate_data["job_position"]) in opening_positions
    # print(critiria_1)
    critiria_2 = clean(candidate_data["company_name"]) == clean("KBTG")
    # print(critiria_2)
    return critiria_1 and critiria_2


In [75]:
def screen_email(email):
    candidate_data = chain.invoke({"email":email})
    is_pass = make_decision(candidate_data)
    if is_pass:
        return "congrats!"
    return "sorry :("

In [88]:
# Test01
screen_email(Frontend_email)

'congrats!'

In [89]:
# Test02
screen_email(chef_email)

'sorry :('

<hr>

## 03 Response email

In [84]:
prompt_tmeplate = ChatPromptTemplate.from_messages(
    [
        ("system",
         """
         You are Hiring Manager named "Khunthong Smith".
         Your task is to write an email response to candidate job application
         """),
        ("user",
         """
         Write the response email in formal English langauge which notify canidate name \"{candidate_name}\" that they \"{result}\"
         """)
    ]
)

model = ChatOpenAI(
    model = "gpt-3.5-turbo",
    temperature = 1.0
)

parser = StrOutputParser()

email_generation_chain = prompt_tmeplate | model | parser

In [86]:
# Test 01
email_generation_chain.invoke(
    {
        "candidate_name":"mr.kasikorn rakthai",
        "result":"Pass"
    }
)

'Subject: Job Application Status - Congratulations!\n\nDear Mr. Kasikorn Rakthai,\n\nI hope this email finds you well. I am writing to inform you that after a careful review of your job application and interview, we are pleased to offer you the position at our company. Congratulations, you have successfully passed the selection process!\n\nWe were impressed with your qualifications, experience, and the passion you demonstrated for the role during the interview. We believe that your skills and expertise will be a valuable addition to our team.\n\nOur team will be in touch shortly to discuss the next steps, including details about the joining date, compensation package, and any other relevant information.\n\nOnce again, congratulations on your successful application. We are looking forward to welcoming you to our organization and working together.\n\nIf you have any questions or need further information, please feel free to reach out to me at [Your Contact Number] or reply to this email.

In [87]:
# Test 02
email_generation_chain.invoke(
    {
        "candidate_name":"mr.aroii dee",
        "result":"Not_Pass"
    }
)

'Subject: Job Application Status Update\n\nDear Mr. Aroii Dee,\n\nI hope this email finds you well. I wanted to personally reach out and notify you about the status of your recent job application with us. After a careful review and assessment of all candidates, we regret to inform you that you have not been selected to proceed to the next stage of the recruitment process.\n\nWe appreciate the time and effort you put into your application and want to thank you for your interest in joining our team. While your qualifications and experience are impressive, we had to make some tough decisions based on specific requirements for the role.\n\nYour application will remain on file with us for future opportunities that match your profile, and we encourage you to apply again in the future if a suitable position becomes available.\n\nWe wish you all the best in your job search and future endeavors. Thank you for considering a career with us.\n\nWarm regards,\n\nKhunthong Smith\nHiring Manager'

<hr>

# 04 Translater

In [90]:
prompt_tmeplate = ChatPromptTemplate.from_messages(
    [
        ("system",
         """
         You are an expert translator proficient in all languages.
         Your task is to translate any email into formal English while maintaining the original meaning and tone.
         """),
        ("user",
         """
         {email}
         """)
    ]
)

model = ChatOpenAI(
    model = "gpt-3.5-turbo",
    temperature = 0.7
)

parser = StrOutputParser()

email_translation_chain = prompt_tmeplate | model | parser

In [91]:
th_email = """
เรียน ผู้จัดการฝ่ายบุคคล,

ข้าพเจ้าเขียนจดหมายนี้เพื่อแสดงความสนใจในการสมัครตำแหน่งหัวหน้าเชฟที่บริษัท KBTG ด้วยประสบการณ์การทำงานด้านการทำอาหารมากกว่า 15 ปี รวมถึงการเป็นเชฟบริหารในร้านอาหารชื่อดังมานานกว่า 5 ปี ข้าพเจ้าเชื่อมั่นในความสามารถที่จะนำพาความสำเร็จมาสู่สถานประกอบการของท่าน

ความเชี่ยวชาญของข้าพเจ้าในอาหารหลากหลายประเภท ความสามารถในการนำทีม และการพัฒนาเมนูที่นวัตกรรมตรงกับวิสัยทัศน์ของ KBTG ข้าพเจ้ารู้สึกตื่นเต้นกับโอกาสที่จะนำความหลงใหลในการทำอาหารที่ยอดเยี่ยมมาสู่ทีมของท่าน

กรุณาพิจารณาประวัติการทำงานที่แนบมานี้ ข้าพเจ้าหวังเป็นอย่างยิ่งว่าจะได้พูดคุยเกี่ยวกับวิธีการที่ประสบการณ์ ทักษะ และใบรับรองต่างๆ ของข้าพเจ้าจะสามารถเป็นประโยชน์ต่อ KBTG

ขอขอบพระคุณที่พิจารณาการสมัครของข้าพเจ้า

ขอแสดงความนับถือ
อร่อย ดี
"""

jp_email = """
採用担当者様

この度、KBTGのヘッドシェフ職に応募させていただきたく、メールをお送りいたしました。15年以上の料理経験と、名高いレストランでの5年間のエグゼクティブシェフとしての経験を持つ私が、御社に貢献できると確信しております。

多様な料理の専門知識、チームリーダーシップ、革新的なメニュー開発のスキルは、KBTGのビジョンに合致していると考えております。料理の卓越性に対する私の情熱を御社のチームに活かせる機会を大変楽しみにしております。

添付した履歴書をご覧いただければ幸いです。私の経歴、スキル、および資格がKBTGにとってどのように役立つかを話し合う機会をいただけることを楽しみにしております。

ご検討の程、よろしくお願い申し上げます。

敬具
アロイ・ディー
"""

In [97]:
# Test01
translation = email_translation_chain.invoke(th_email)
print(translation)

Dear Human Resources Manager,

I am writing this letter to express my interest in applying for the Head Chef position at KBTG Company, with over 15 years of experience in food preparation, as well as over 5 years of experience as an executive chef in a renowned restaurant. I am confident in my ability to lead success into your establishment.

My expertise in a variety of cuisines, team leadership skills, and innovation in menu development align with KBTG's vision. I am excited about the opportunity to bring my passion for creating exceptional dishes to your team.

Please consider the attached resume for my work history. I am hopeful that a discussion regarding how my experience, skills, and certifications can benefit KBTG will be possible.

Thank you for considering my application.

Sincerely,
Aroi Dee


In [98]:
# Test02
translation = email_translation_chain.invoke(jp_email)
print(translation)

Dear Hiring Manager,

I am writing to express my interest in the Head Chef position at KBTG. With over 15 years of culinary experience, including 5 years as an Executive Chef at renowned restaurants, I am confident that I can contribute effectively to your organization.

I believe that my diverse culinary expertise, team leadership skills, and innovative menu development abilities align well with KBTG's vision. I am looking forward to the opportunity to bring my passion for culinary excellence to your team.

I have attached my resume for your review. I am excited about the prospect of discussing how my background, skills, and qualifications can benefit KBTG.

Thank you for considering my application.

Sincerely,
Aloy Dee


<hr>

In [ ]:
from typing import Dict

# Step 01: Email Data Extraction
email_extraction_prompt = ChatPromptTemplate.from_messages(
    [
        ("system",
         """
         You are a Hiring Manager at KBTG reading a candidate's email.
         A candidate's email text will be given.
         Your task is to identify the following information and convert it into a JSON object with these keys:
         - job_position : string, the job position which the candidate is applying for.
         - company_name : string, the company name which the candidate is applying for.
         - candidate_name : string, the candidate name
         Answer in the form of a JSON object
         """),
        ("user",
         "{email}")
    ]
)

email_extraction_model = ChatOpenAI(
    model="gpt-3.5-turbo",
    temperature=0.0
)

email_extraction_parser = JsonOutputParser()
email_extraction_chain = email_extraction_prompt | email_extraction_model | email_extraction_parser

# Step 02: Pass or NotPass Decision
def clean(text):
    return text.lower().strip().replace(" ", "")

def make_decision(candidate_data: Dict[str, str]) -> bool:
    opening_positions = ["Frontend Developer", "Backend Developer", "AI Developer", "Blockchain Developer"]
    opening_positions = set([clean(e) for e in opening_positions])
    criteria_1 = clean(candidate_data["job_position"]) in opening_positions
    criteria_2 = clean(candidate_data["company_name"]) == clean("KBTG")
    return criteria_1 and criteria_2

# Step 03: Response Email Generation
response_email_prompt = ChatPromptTemplate.from_messages(
    [
        ("system",
         """
         You are a Hiring Manager named "Khunthong Smith".
         Your task is to write an email response to a candidate's job application.
         """),
        ("user",
         """
         Write the response email in formal English language which notifies candidate name \"{candidate_name}\" that they \"{result}\".
         """)
    ]
)

response_email_model = ChatOpenAI(
    model="gpt-3.5-turbo",
    temperature=1.0
)

response_email_parser = StrOutputParser()
response_email_chain = response_email_prompt | response_email_model | response_email_parser

# Full Pipeline Function
def process_email_pipeline(email: str) -> str:
    # Step 01: Extract Data
    candidate_data = email_extraction_chain.invoke({"email": email})
    
    # Step 02: Make Decision
    is_pass = make_decision(candidate_data)
    result = "Pass" if is_pass else "Not_Pass"
    
    # Step 03: Generate Response Email
    response_email = response_email_chain.invoke(
        {
            "candidate_name": candidate_data["candidate_name"],
            "result": result
        }
    )
    return response_email

# Example Emails for Testing
frontend_email = """
Dear Hiring Manager,

My name is Kasikorn Rakthai, and I am writing to express my interest in the Frontend Developer position at KBTG. I believe my skills and experience make me a strong candidate for this role.

Best regards,
Kasikorn Rakthai
"""

chef_email = """
Dear Hiring Manager,

My name is Aroii Dee, and I am applying for the position of Chef at KBTG. Cooking has been my lifelong passion, and I believe I can contribute to your company's success.

Sincerely,
Aroii Dee
"""

# Test Pipeline
print(process_email_pipeline(frontend_email))  # Expected: Congrats email
print(process_email_pipeline(chef_email))      # Expected: Sorry email
